In [1]:
import os
import sys
import ipywidgets as widgets
import pyvo
from IPython.display import Image, display, clear_output
import pandas as pd
import glue_jupyter as gj
from astropy.io.votable import from_table, writeto
import base64
from IPython.core.display import HTML
sys.path.insert(0, os.path.abspath('./'))
from final_core import QueryBuilder

query_out = widgets.Output(layout=widgets.Layout(width='100%'))
query_out.layout.border = "2px solid green"
table_out = widgets.Output(layout=widgets.Layout(width='100%'))
graph_out = widgets.Output(layout=widgets.Layout(width='100%')) 
query_builder = QueryBuilder()



def graph(df):
    with graph_out:
        clear_output()
        app = gj.jglue()
        out_data = app.load_data('output_file.csv')
        columns = df.columns
        scatter_viewer = app.scatter2d(x= columns[0],
                                       y= columns[1],
                                       data=out_data,
                                       show=True)
        display(scatter_viewer)
        

def Search_button_result(b):
    with table_out:
        clear_output()
        b.disabled = True
        result_table = query_builder.search_query()
        df = result_table.to_table().to_pandas()
        ####### add the download button here  #########
        ### the button allows user to download the table to csv file ######
        b64 = base64.b64encode(df.to_string().encode())
        payload = b64.decode()
        html = f"""<a download="testfile" href="data:text;base64, {payload}" target="_blank">Download File</a>"""
        display(HTML(html))
        display(df)
        df.to_csv('output_file.csv', index=False)
        graph(df)
        b.disabled = False
        

button = widgets.Button(
            description="SEARCH",
            layout=widgets.Layout(flex='1 1 auto',
                                  width='auto'),
            style=widgets.ButtonStyle(button_color='#C8F7FD'))

button.on_click(Search_button_result)


query_builder.list_test.append(button)

with query_out:
    query_builder.Start_query()

###########################################
tab = widgets.Tab()
tab_contents = ['Query', 'Result', 'Plot']
children = [query_out, table_out, graph_out]
tab.children = children
for i in range(len(children)):
    tab.set_title(i, tab_contents[i])

display(tab)


